In [0]:
sql = """
CREATE VOLUME IF NOT EXISTS test_catalog.landing.csv_staging_volume;
"""
spark.sql(sql)

dbutils.fs.cp(
    "dbfs:/mnt/dbacademy-datasets/data-engineer-learning-path/v02/ecommerce/raw/csv_staging",
    "/Volumes/test_catalog/landing/csv_staging_volume/sales",
    recurse=True,
)

In [0]:
sql = """
CREATE VOLUME IF NOT EXISTS test_catalog.landing.csv_source_volume
"""
spark.sql(sql)

dbutils.fs.cp(
    "dbfs:/mnt/dbacademy-datasets/data-engineer-learning-path/v02/ecommerce/raw/sales-csv",
     "/Volumes/test_catalog/landing/csv_source_volume/sales",
    recurse=True
)

In [0]:
sql = """
CREATE VOLUME IF NOT EXISTS test_catalog.landing.csv_checkpoint_volume
"""
spark.sql(sql)

In [0]:
%sql
DROP TABLE IF EXISTS test_catalog.bronze.python_csv_autoloader;

In [0]:
checkpoint_file_location = "/Volumes/test_catalog/landing/csv_checkpoint_volume/sales"
src_file_location = "/Volumes/test_catalog/landing/csv_source_volume/sales"

# load data
(spark
    .readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "csv")
        .option("header", "true")
        .option("sep", "|")
        .option("inferSchema", "true")
        .option("cloudFiles.schemaLocation", f"{checkpoint_file_location}")
        .load(f"{src_file_location}")
    .writeStream
        .format("delta")
        .option("checkpointLocation", f"{checkpoint_file_location}")
        .trigger(once=True)
        .outputMode("Append")
        .toTable("test_catalog.bronze.python_csv_autoloader")
)

In [0]:
%sql
DESCRIBE HISTORY test_catalog.bronze.python_csv_autoloader;

In [0]:
target = '/Volumes/test_catalog/landing/csv_source_volume'
file_1 = '/Volumes/test_catalog/landing/csv_staging_volume/001.csv'
file_2 = '/Volumes/test_catalog/landing/csv_staging_volume/002.csv'
file_3 = '/Volumes/test_catalog/landing/csv_staging_volume/003.csv'

dbutils.fs.cp(file_1, target)